## - **Importando Bibliotecas**

In [22]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

## - **Entrando no Indeed**

In [23]:
driver = webdriver.Chrome()
url_login = "https://br.indeed.com/"
driver.get(url_login)

## - **Pequisando Vagas**

In [24]:
driver.get(url_login)

try:
    recusar_cookies = driver.find_element(By.ID, "onetrust-reject-all-handler")
    recusar_cookies.click()
except:
    pass


input_cargo = "Desenvolvedor Python"
input_localidade = "Minas Gerais"


localidade = driver.find_element(By.ID, "text-input-where")
cargo = driver.find_element(By.ID, "text-input-what")


cargo.send_keys(Keys.CONTROL + "a")
cargo.send_keys(Keys.BACKSPACE)

sleep(1)

localidade.send_keys(Keys.CONTROL + "a")
localidade.send_keys(Keys.BACKSPACE)

sleep(1)

cargo.send_keys(input_cargo)
localidade.send_keys(input_localidade)
localidade.send_keys(Keys.ENTER)

## - **Definindo Função de Descrições de Vaga**

In [25]:
def raspar_dados_vaga():
    try:
        nome = driver.find_element(By.CLASS_NAME, "css-hon9z8.eu4oa1w0")
        print("Nome Empresa: ", nome.text, "\n")
    except:
        pass

    try:
        jobdescription = driver.find_element(By.ID, "jobDescriptionText")
        print(jobdescription.text)
    except:
        pass

    try:
        benefits = driver.find_element(By.ID, "benefits")
        print(benefits.text)
    except:
        pass

    try:
        jobdetails = driver.find_element(By.ID, "jobDetailsSection")
        print(jobdetails.text)
    except:
        pass

    try:
        location = driver.find_element(By.ID, "jobLocationWrapper")
        print(location.text)
    except:
        pass

## - **Iterando sobre Vagas**

In [26]:
def iterar_vagas():
    ul_element = driver.find_element(By.CSS_SELECTOR, ".css-zu9cdh.eu4oa1w0")


    try:
        ignore = ul_element.find_element(By.ID, "mosaic-afterFifthJobResult")
        
        ignore_hamburguer = ignore.find_element(By.ID, "menu-button--menu--1")
        ignore_hamburguer.click()

        fechar = driver.find_element(By.CLASS_NAME, "css-1d4wshm.ehvvxyn1")
        fechar.click()

    except:
        pass

    vagas = ul_element.find_elements(By.CLASS_NAME, "css-5lfssm.eu4oa1w0")

    print(len(vagas))

    for i, vaga in enumerate(vagas):

        if i == 5:
            continue

        if i == 11:
            continue

        if i == 17:
            continue

        print(i)

        vaga.click()

        sleep(5)

        raspar_dados_vaga()

        sleep(1.5)


In [27]:
for i in range(5):
    url = f"https://br.indeed.com/jobs?q=Desenvolvedor+Python&l=Minas+Gerais&start={i*10}"
    driver.get(url)
    sleep(2)

    try:
        botao = driver.find_element(By.CLASS_NAME, "css-yi9ndv.e8ju0x51")
        botao.click()
    except:
        pass

    iterar_vagas()

18
0
Nome Empresa:  BairesDev 


1
Nome Empresa:  BairesDev 

Há mais de 15 anos, a BairesDev® é líder em projetos tecnológicos. Entregamos soluções inovadoras para gigantes como Google e Rolls-Royce e as startups mais inovadoras do Vale do Silício.

Temos uma equipe diversa de mais de 4.000 profissionais formada pelo Top 1% dos talentos de tecnologia do mundo, trabalhando remotamente em cargos de impacto global.

Ao se candidatar para este cargo, você está dando o primeiro passo em um processo que vai além do comum. Nosso objetivo é alinhar suas paixões, habilidades, e expectativas às nossas vagas, colocando você no caminho para desenvolver uma carreira de sucesso excepcional.
2
Nome Empresa:  DVL Distribuidora via Lactea 

ANALISTA - DESENVOLVEDOR FULL STACK
Estamos em busca de um desenvolvedor full stack para integrar nossa equipe.
Responsabilidades:
Skills:
- Conhecimento em pelo menos 1 das linguagens: JAVA, C#, PHP, RUBY, PYTHON.
- Conhecimento sólidos em banco de dados ( Trigger